In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define file path and parameters
fileFullPath = r'D:\Drone-Swarm-Detection-with-AWR2243\Our data\Radar_Data\waruna_drone\waruna_forward_backward\master_0000_data.bin';
frameIdx = 50# Index of the frame to read
numSamplePerChirp = 256  # Number of samples per chirp
numChirpPerLoop = 12  # Number of chirps per loop
numLoops = 128 # Number of loops per frame
numRXPerDevice = 4  # Number of receiving channels per device
numDevices = 4  # Number of devices in the cascade (if needed)

# Function to read binary radar data
def readBinFile(fileFullPath, frameIdx, numSamplePerChirp, numChirpPerLoop, numLoops, numRXPerDevice):
    Expected_Num_SamplesPerFrame = numSamplePerChirp * numChirpPerLoop * numLoops * numRXPerDevice * 2
    with open(fileFullPath, 'rb') as fp:
        # Move to the desired frame in the file
        fp.seek((frameIdx - 1) * Expected_Num_SamplesPerFrame * 2)
        adcData1 = np.fromfile(fp, dtype=np.uint16, count=Expected_Num_SamplesPerFrame)

    # Convert the 16-bit data to signed integers
    neg = (adcData1 >> 15) == 1  # Check the sign bit
    adcData1 = adcData1.astype(np.int32)
    adcData1[neg] -= 2**16

    # Combine the I and Q channels into complex values
    adcData1 = adcData1[0::2] + 1j * adcData1[1::2]

    # Reshape and permute the data
    adcData1Complex = np.reshape(adcData1, (numRXPerDevice, numSamplePerChirp, numChirpPerLoop, numLoops), order='F')
    adcData1Complex = np.transpose(adcData1Complex, (1, 3, 0, 2))  # Permute dimensions
    return adcData1Complex


for frameIdx in range(1,128):

    adcDataComplex = readBinFile(fileFullPath, frameIdx, numSamplePerChirp, numChirpPerLoop, numLoops, numRXPerDevice)
    average_array=[]
    # Select antenna index and extract chirp ADC matrix
    for antennaIdx in range (0,4):
        chirp_ADC_matrix = adcDataComplex[:, :, antennaIdx, :]
        # Assuming you want the data from the first transmitter (index 0)
        specific_transmitter_data = chirp_ADC_matrix[:, :, 0]
        specific_transmitter_data=np.transpose(specific_transmitter_data)
        #specific_transmitter_data =filtfilt(b,a,specific_transmitter_data , axis=0)
        print(specific_transmitter_data.shape)  # Should print (256, 128)



        # Extract first chirp of the first loop
        first_chirp_first_loop = chirp_ADC_matrix[:, 0, 0]

        # Initialize an empty list to store each chirp
        all_chirps = []
        # Loop over loops and chirps per loop
        for i in range(numLoops):
            for j in range(numChirpPerLoop):
                # Append each chirp to the list
                all_chirps.append(chirp_ADC_matrix[:, i, j])

        # Convert the list to a numpy array and reshape it to (768, 256)
        all_chirps = np.array(all_chirps).reshape(256,1536)


        #chirps_matrix =all_chirps 
        chirps_matrix=specific_transmitter_data
    
        chirps_matrix=np.transpose(chirps_matrix)
    
        # Constants (adjust based on your radar parameters)
        fc = 77e9  # Radar operating frequency (77 GHz for mmWave radar)
        c = 3e8  # Speed of light (m/s)
        sweepBandwidth = 0.89e9  # Bandwidth of the FMCW radar sweep (3.16 GHz)
        chirpDuration = 30e-6  # Chirp duration (40 microseconds)

        Nfft_range = 300  # Number of FFT points for range dimension
        Nfft_doppler = 97# Number of FFT points for Doppler dimension

        # Perform 2D FFT along both the range (ADC samples) and Doppler (chirps) dimensions
        range_fft = np.fft.fft(chirps_matrix, Nfft_range, axis=0)  # FFT across range (ADC samples)
        doppler_fft = np.fft.fftshift(np.fft.fft(range_fft, Nfft_doppler, axis=1), axes=1)  # FFT across Doppler (chirps), with shift
        print("doppler_fft matrix diamension is", doppler_fft.shape )
        print(np.mean(doppler_fft))
        average_array.append(doppler_fft)
        # Calculate the range and velocity axis values
        range_res = c / (2 * sweepBandwidth)  # Range resolution (meters)
        max_range = range_res * (Nfft_range - 1)  # Maximum measurable range
        range_axis = np.linspace(0, max_range, Nfft_range)  # Range axis for plotting

        doppler_res = 1 / (numChirpPerLoop * chirpDuration)  # Doppler resolution (Hz)
        max_doppler = doppler_res * (Nfft_doppler / 2)  # Maximum Doppler shift (Hz)
    
        # Doppler axis for plotting (ensure symmetry around 0)
        doppler_axis = np.linspace(-max_doppler, max_doppler, Nfft_doppler)

        # Convert Doppler frequency to velocity (m/s)
        velocity_axis = doppler_axis * (c / (2 * fc))  # Velocity axis using Doppler shift

        # # Plot the Range-Velocity map (absolute value of FFT)
        # plt.figure()
        # plt.imshow(20 * np.log10(np.abs(doppler_fft)), aspect='auto', extent=[velocity_axis[0], velocity_axis[-1], range_axis[0], range_axis[-1]], origin='lower',cmap='jet')
        # plt.xlabel('Velocity (m/s)')
        # plt.ylabel('Range (m)')
        # plt.title('Range-Velocity Map')
        # plt.colorbar()
    print(len(average_array))
        # Plot the Range-Velocity map (absolute value of FFT)
    # Calculate the mean of the four elements


    # Calculate the mean across the first axis (i.e., across the 4 arrays)
    mean_array = np.mean(average_array, axis=0)

    # Apply the log scale transformation to the mean
    #result matrix diamension is (300, 97)
    result = 20 * np.log10(np.abs(mean_array))
    result_without_process =  result
    mean_value=np.mean(result)
    max_value=np.max(result)
    print("Max value of matrix",max_value)
    # Print the mean of the result matrix
    print("Mean of the result matrix:", np.mean(result))

    # Loop through the result matrix and apply the condition
    for i in range(result.shape[0]):  # Loop over rows
        for j in range(result.shape[1]):  # Loop over columns
            if 60 < result[i, j] < 86:
                # Replace result[i, j] with the corresponding mean value
                result[i, j] = mean_value
    # Set all values in the 49th column to this mean value
    result[:,48] = mean_value
    plt.figure()
    plt.imshow(result_without_process, aspect='auto', extent=[velocity_axis[0], velocity_axis[-1], range_axis[0], range_axis[-1]], origin='lower',cmap='jet')
    plt.xlabel('Velocity (m/s)')
    plt.ylabel('Range (m)')
    plt.title('Range-Velocity Map')
    plt.colorbar()